**Please UPVOTE for Encouragement**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings 
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train=pd.read_csv('../input/train.csv')
df_train.shape

In [ ]:
df_test=pd.read_csv('../input/test.csv')
df_test.shape

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.columns

In [ ]:
df_test.columns

In [ ]:
df_train.dtypes

In [ ]:
df_train['SalePrice']=np.log1p(df_train['SalePrice'])
sns.distplot(df_train['SalePrice'],fit=norm)

In [ ]:
train_null=df_train.isnull().sum()
train_null=train_null[train_null>0].sort_values(ascending=False)
train_null

In [ ]:
test_null=df_test.isnull().sum()
test_null=test_null[test_null>0].sort_values(ascending=False)
test_null

In [ ]:
plt.figure(figsize=(32,32)) 
sns.heatmap(df_train.corr(),vmin=0, annot=True, cbar=True, cmap="RdYlGn")

In [ ]:
corrmat=df_train.corr()
top_features=corrmat.index[abs(corrmat['SalePrice'])>0.5]
plt.figure(figsize=(12,12)) 
sns.heatmap(df_train[top_features].corr(),vmin=0, annot=True, cbar=True, cmap="RdYlGn")

In [ ]:
sns.pairplot(df_train, hue="SalePrice")

In [ ]:
categorical_feat=df_train.select_dtypes(include=['object']).columns
categorical_feat

In [ ]:
numerical_feat=df_train.select_dtypes(exclude=['object']).columns
numerical_feat

In [ ]:
numercial_feat=numerical_feat.drop('SalePrice')

In [ ]:
str(len(categorical_feat)),str(len(numerical_feat))

In [ ]:
train_num = df_train[numerical_feat]
train_cat = df_train[categorical_feat]

In [ ]:
train_num.isnull().values.sum()

In [ ]:
train_num=train_num.fillna(train_num.median())
train_num.isnull().values.sum()

In [ ]:
from scipy.stats import skew
skewness=train_num.apply(lambda x:skew(x))
skewness=skewness[abs(skewness>0.5)].index
skewness.sort_values(ascending=False)

In [ ]:
skew_features=df_train[skewness]
skew_features.columns

In [ ]:
skew_features = np.log1p(skew_features)

In [ ]:
train_cat.shape 

In [ ]:
train_cat=pd.get_dummies(train_cat)
train_cat.shape

In [ ]:
train_cat.isnull().values.sum()

In [ ]:
train = pd.concat([train_cat,train_num],axis=1)
train.shape

In [ ]:
x=train
y=df_train.SalePrice

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_cv,y_train,y_cv=train_test_split(x,y,test_size=0.3)

In [ ]:
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
model1=XGBRegressor(booster='gblinear', reg_lambda=2,reg_alpha=1,cv=10)
model1.fit(x_train,y_train)
pred_cv=model1.predict(x_cv)

In [ ]:
rmse_CV_train(model1).mean()

In [ ]:
pred_test=model1.predict(df_test)

In [ ]:
submission=pd.read_csv('../input/sample_submission.csv')

In [ ]:
submission['SalePrice']=pred_test
submission['Id']=df_test['Id']
pd.DataFrame(submission,columns=['Id','SalePrice']).to_csv('xgboostregression.csv',index=False)